In [1]:

import os

os.environ["KERAS_BACKEND"] = "tensorflow"
from preprocessing import getdataset,character_to_number, number_to_character

MIN_FREQ = 20

dataset , table, MAX_LEN = getdataset()
table.crop(MIN_FREQ)
print(table.vocab_size)


2024-12-27 16:16:21.238152: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-27 16:16:21.248713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735312581.261711   97372 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735312581.265862   97372 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-27 16:16:21.279403: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

shocolat
6392


In [2]:
from network import make_model

EMBED_DIM = 256
NUM_HEADS = 3
NUM_BLOCS = 4
hidden_dim = 256

model = make_model(MAX_LEN, table.vocab_size, EMBED_DIM, NUM_HEADS,NUM_BLOCS,hidden_dim)

I0000 00:00:1735312583.348658   97372 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4568 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
/home/celeste/miniconda3/envs/imm/lib/python3.12/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'token_and_position_embedding' (of type TokenAndPositionEmbedding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 62)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masking (Masking)               │ (None, 62)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, 62, 256)        │     1,652,224 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ (None, 62, 256)        │       855,808 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_1             │ (None, 62, 256)        │       855,808 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_2             │ (None, 62, 256)        │       855,808 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_3             │ (None, 62, 256)        │       855,808 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 62, 256)        │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 1, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1, 6392)        │     1,642,744 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,783,992 (25.88 MB)

 Trainable params: 6,783,992 (25.88 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
def custom_loss(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    loss *= mask
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)

model.compile(optimizer="adam",
              loss=custom_loss,
              metrics=["sparse_categorical_accuracy"])


In [5]:
from preprocessing import Dataloader

dl = Dataloader(dataset, table, MAX_LEN,batch_size=128)  

tf_dataset = tf.data.Dataset.from_generator(
        dl.get_generator,
        output_signature=(
            tf.TensorSpec(shape=(None, MAX_LEN), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 1), dtype=tf.float32)
        )
    )



In [6]:
history = model.fit(tf_dataset ,epochs=30, batch_size=128,verbose=1)
model.save("model.h5")

Epoch 1/30


/home/celeste/miniconda3/envs/imm/lib/python3.12/site-packages/keras/src/layers/layer.py:932: UserWarning: Layer 'embedding' (of type Embedding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
I0000 00:00:1735312594.784948   97481 service.cc:148] XLA service 0x76ca48002190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735312594.784967   97481 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-12-27 16:16:34.907371: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1735312595.184324   97481 assert_op.cc:38] Ignoring Assert operator compile_loss/custom_loss/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Ass

      2/Unknown 17s 79ms/step - loss: 8.7779 - sparse_categorical_accuracy: 0.0039   

2024-12-27 16:16:40.982162: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'copy_fusion', 608 bytes spill stores, 608 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 52 bytes spill stores, 52 bytes spill loads

I0000 00:00:1735312601.012298   97481 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   5614/Unknown 379s 64ms/step - loss: nan - sparse_categorical_accuracy: 0.0139

In [ ]:
history = model.fit(X, Y, epochs=30, batch_size=128,verbose=1)
model.save("model_2.h5")

In [ ]:
model.save("model_3.h5")

In [ ]:
#text_to_begin_with = "".split(" ")
text_to_begin_with = []
begining = [ table.table[i] for i in text_to_begin_with ]
nb_to_repeat = MAX_LEN - len(begining) -1
input = np.array([[2] + begining + [0] * nb_to_repeat])
exit = False
nb_iter = len(text_to_begin_with)+1
max_iter = MAX_LEN

print( "".join( [table.number_to_character( input[0,i] ) for i in range(len(input[0]))] ) )

while not exit:
    output = model.predict(input)

    input[0,nb_iter] = np.argmax(output[0,nb_iter-1])
    nb_iter += 1
    if nb_iter == max_iter:
        exit = True
    
print( "".join( [table.number_to_character( input[0,i] ) for i in range(len(input[0]))] ) )

<sos><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━